In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
## Drop irrelevant columns
data=data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

In [4]:
## Encode categorical variables
label_encoder_gender=LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])

In [5]:
## Onehot encode "Geography"
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder(sparse_output=False)
geo_encoder=onehot_encoder_geo.fit_transform(data[["Geography"]])

In [6]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

In [7]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop("Geography", axis=1),geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
## Save the encoders
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file)

In [8]:
## Divide the dataset into Independent and Dependent features
X=data.drop("EstimatedSalary",axis=1)
y=data["EstimatedSalary"]

## Split the data in training and testing sets
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2,random_state=42)

## Scale these features

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
## Save the scaler

with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

### ANN Regression Implementation

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
## Build Our ANN Model
model=Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)), ##HL1 Connected with input layer
    Dense(32,activation="relu"), ## HL2
    Dense(1) ## Output layer for regression (linear regression is default)
])

## compile the model
model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["mae"])
model.summary()

c:\Users\peke5\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
## Set ip the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir="regression/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [16]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [17]:
## Training the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100373.0469 - mae: 100373.0469 - val_loss: 98500.3047 - val_mae: 98500.3047
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99579.0703 - mae: 99579.0703 - val_loss: 96914.8828 - val_mae: 96914.8828
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 96832.1406 - mae: 96832.1406 - val_loss: 92908.9922 - val_mae: 92908.9922
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 91465.1016 - mae: 91465.1016 - val_loss: 86279.8984 - val_mae: 86279.8984
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 83650.4453 - mae: 83650.4453 - val_loss: 77732.1484 - val_mae: 77732.1484
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 74584.7266 - mae: 74584.7266 - val_loss: 68935.9453 - val_mae: 68935.9453
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 65918.7266 - mae: 65918.7266 - val_loss: 61363.4023 - val_mae: 61363.4023
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [18]:
## Load Tensorboard Extension 
%load_ext tensorboard

In [20]:
%tensorboard --logdir regression/logs/fit/20250811-163632/

Reusing TensorBoard on port 6006 (pid 10964), started 0:00:09 ago. (Use '!kill 10964' to kill it.)

In [21]:
## Evaluate model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50376.2227 - mae: 50376.2227
Test MAE: 50376.22265625


In [22]:
model.save("regression_model.h5")